# Personalize with temporal evaluation on hold-out set

The notebook largely follows the basic notebook, with the additional tweak to hold-out 10% of "future" data for every user. Then, we set up an inference endpoint to bring recommendation and evaluate externally on the held-out data.

The other minor difference is that we predict views rather than ratings. We find it more interesting to demonstrate recommendation of unpopular (yet highly personalized) movies than to recommend popular movies that everyone would enjoy (without personalization needs).

In [1]:
import boto3, os
import json
import numpy as np
import pandas as pd
import time

In [2]:
os.environ['AWS_DEFAULT_REGION']="us-east-1"

In [3]:
suffix = str(np.random.uniform())[4:9]

In [4]:
bucket = "demo-temporal-holdout-"+   suffix        # replace with the name of your S3 bucket
filename = "DEMO-temporal-holdout.csv"

In [5]:
!aws s3 mb s3://{bucket}

make_bucket: demo-temporal-holdout-79580


In [6]:
!curl -O https://s3-us-west-2.amazonaws.com/personalize-cli-json-models/personalize.json
!curl -O https://s3-us-west-2.amazonaws.com/personalize-cli-json-models/personalize-runtime.json
!aws configure add-model --service-model file://`pwd`/personalize.json --service-name personalize
!aws configure add-model --service-model file://`pwd`/personalize-runtime.json --service-name personalize-runtime

personalize = boto3.client(service_name='personalize', endpoint_url='https://personalize.us-east-1.amazonaws.com')
personalize_runtime = boto3.client(service_name='personalize-runtime', endpoint_url='https://personalize-runtime.us-east-1.amazonaws.com')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 46104  100 46104    0     0   437k      0 --:--:-- --:--:-- --:--:--  437k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3320  100  3320    0     0  97647      0 --:--:-- --:--:-- --:--:-- 97647


# Download and process benchmark data

In [7]:
!curl -O http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5778k  100 5778k    0     0  9912k      0 --:--:-- --:--:-- --:--:-- 9912k
Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [8]:
data = pd.read_csv('./ml-1m/ratings.dat', sep='::', names=['USER_ID','ITEM_ID','RATING','TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,1,1193,5,978300760
1,1,661,3,978302109
...,...,...,...,...
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


### Extract last 10% of interactions per user as hold-out tests

In [9]:
ranks = data.groupby('USER_ID').TIMESTAMP.rank(pct=True, method='first')
data = data.join((ranks>0.9).to_frame('holdout'))
holdout = data[data['holdout']].drop('holdout', axis=1)
data = data[~data['holdout']].drop('holdout', axis=1)

In [10]:
# data = data[data['RATING'] > 3.6]  # Use all data to predict view recommendations
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
print('unique users %d; unique items %d'%(
    len(data['USER_ID'].unique()), len(data['ITEM_ID'].unique())))

unique users 6040; unique items 3684


### Upload data

In [11]:
data.to_csv(filename, index=False)
!head {filename}

boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

USER_ID,ITEM_ID,TIMESTAMP
1,1193,978300760
1,661,978302109
1,914,978301968
1,3408,978300275
1,1197,978302268
1,1287,978302039
1,2804,978300719
1,594,978302268
1,919,978301368


# Create Schema

In [12]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "DEMO-temporal-schema-"+suffix,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:261294318658:schema/DEMO-temporal-schema-79580",
  "ResponseMetadata": {
    "RequestId": "27540caa-6a69-4a16-ab3c-a8daeeb271e7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 01 Jun 2019 01:53:58 GMT",
      "x-amzn-requestid": "27540caa-6a69-4a16-ab3c-a8daeeb271e7",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


# Create and Wait for Dataset Group

In [13]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "DEMO-temporal-dataset-group-"+suffix
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:261294318658:dataset-group/DEMO-temporal-dataset-group-79580",
  "ResponseMetadata": {
    "RequestId": "ae62729f-b9d8-4a16-9805-ac06d73b4148",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 01 Jun 2019 01:54:10 GMT",
      "x-amzn-requestid": "ae62729f-b9d8-4a16-9805-ac06d73b4148",
      "content-length": "112",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [14]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


In [16]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn,
    name = "DEMO-temporal-dataset-"+suffix
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:261294318658:dataset/DEMO-temporal-dataset-group-79580/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "069e861a-9e98-4743-8e16-19712dd87a3f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 01 Jun 2019 01:55:07 GMT",
      "x-amzn-requestid": "069e861a-9e98-4743-8e16-19712dd87a3f",
      "content-length": "114",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


# Prepare, Create, and Wait for Dataset Import Job

In [17]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy));

### Create S3 Read-Only Access Role

In [18]:
from botocore.exceptions import ClientError

In [19]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise
print(role_arn)

arn:aws:iam::261294318658:role/PersonalizeS3Role-79580


In [20]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-temporal-dataset-import-job-"+suffix,
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:261294318658:dataset-import-job/DEMO-temporal-dataset-import-job-79580",
  "ResponseMetadata": {
    "RequestId": "0926a9d0-a568-4658-9fde-19a9d45cca10",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 01 Jun 2019 01:55:43 GMT",
      "x-amzn-requestid": "0926a9d0-a568-4658-9fde-19a9d45cca10",
      "content-length": "126",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for Dataset Import Job and Dataset Import Job Run to Have ACTIVE Status

In [21]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


# Create Solution

In [22]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims


There are many recipes for different scenarios. In this example, we only have interactions data, so we will choose one from the basic recipes.

| Feasible? | Recipe | Description 
|-------- | -------- |:------------
| Y | aws-popularity-count | Calculates popularity of items based on count of events against that item in user-item interactions dataset.
| Y | aws-hrnn | Predicts items a user will interact with. A hierarchical recurrent neural network which can model the temporal order of user-item interactions.
| N - requires meta data | aws-hrnn-metadata | Predicts items a user will interact with. HRNN with additional features derived from contextual (user-item interaction metadata), user medata (user dataset) and item metadata (item dataset)
| N - for bandits and requires meta data | aws-hrnn-coldstart | Predicts items a user will interact with. HRNN-metadata with with personalized exploration of new items.
| N - for item-based queries | aws-sims | Computes items similar to a given item based on co-occurrence of item in same user history in user-item interaction dataset
| N - for reranking a short list | aws-personalized-ranking | Reranks a list of items for a user. Trains on user-item interactions dataset. 


We (or autoML) can run all of these basic recipes and choose the best-performing model from internal metrics. We recommend comparisons, especially with popularity-baseline, to see the lifts in metrics via personalization. However, in this demo, we will pick one recipe - aws-hrnn, to focus on external evaluations.

In [23]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

In [24]:
create_solution_response = personalize.create_solution(
    name = "DEMO-temporal-solution-"+suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:261294318658:solution/DEMO-temporal-solution",
  "ResponseMetadata": {
    "RequestId": "c8efe42e-5694-48fa-b93a-913e33f3e4db",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 01 Jun 2019 02:16:22 GMT",
      "x-amzn-requestid": "c8efe42e-5694-48fa-b93a-913e33f3e4db",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [26]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:261294318658:solution/DEMO-temporal-solution/b0644545",
  "ResponseMetadata": {
    "RequestId": "ca50282c-94e1-4d8b-9e67-be92df795442",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 01 Jun 2019 02:17:22 GMT",
      "x-amzn-requestid": "ca50282c-94e1-4d8b-9e67-be92df795442",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for Solution Version to Have ACTIVE Status

In [27]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_P

### Get Metrics of Solution

In [32]:
get_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:261294318658:solution/DEMO-temporal-solution/b0644545",
  "metrics": {
    "coverage": 0.5541,
    "mean_reciprocal_rank_at_25": 0.0937,
    "normalized_discounted_cumulative_gain_at_10": 0.1361,
    "normalized_discounted_cumulative_gain_at_25": 0.1744,
    "normalized_discounted_cumulative_gain_at_5": 0.1111,
    "precision_at_10": 0.0216,
    "precision_at_25": 0.0148,
    "precision_at_5": 0.0285
  },
  "ResponseMetadata": {
    "RequestId": "a9afd35b-e422-4e1e-879d-3cda46d71bc5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 01 Jun 2019 03:10:38 GMT",
      "x-amzn-requestid": "a9afd35b-e422-4e1e-879d-3cda46d71bc5",
      "content-length": "406",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


# Create and Wait for Campaign

In [33]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-temporal-campaign-"+suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 2,    
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:261294318658:campaign/DEMO-temporal-campaign-79580",
  "ResponseMetadata": {
    "RequestId": "4fae1400-1742-4bec-823f-47c2992ea22e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 01 Jun 2019 03:11:31 GMT",
      "x-amzn-requestid": "4fae1400-1742-4bec-823f-47c2992ea22e",
      "content-length": "98",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for Campaign to Have ACTIVE Status

In [35]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: ACTIVE


# Evaluate using external metrics

An explanation of the evaluation metrics are provided at https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html

For example, suppose we recommend four items and two of them are relevant, $r=[0,1,0,1]$. In this case, the metrics are:

|Name	|Example	|Explanation
|:------|:----------|:----------
|Precision@K	|$\frac{2}{4} = 0.5$	|Total relevant items divided by total recommended items.
|Mean reciprocal ranks (MRR@K)	|${\rm mean}(\frac{1}{2} + \frac{1}{4}) = 0.375$	|Considers positional effects by computing the mean of the inverse positions of all relevant items.
|Normalized discounted cumulative gains (NDCG@K)	|$\frac{\frac{1}{\log(1 + 2)} + \frac{1}{\log(1 + 4)}}{\frac{1}{\log(1 + 1)} + \frac{1}{\log(1 + 2)}} = 0.65$	|Considers positional effects by applying inverse logarithmic weights based on the positions of relevant items, normalized by the largest possible scores from ideal recommendations.
|Average precision (AP@K)	|${\rm mean}(\frac{1}{2} + \frac{2}{4}) = 0.5$	|Average precision@K where K is the position of every relevant item.

These metrics are different from the internal metrics in two aspects:
* They are evaluated at different times, which may imply different click rates. We recommend to always keep the evaluations in the same time periods to avoid temporal drifts.
* The example external evaluations may hold out and consider multiple items as ground truth, while the internal evaluations only hold out the last item in each user-history as the ground truth. There is no absolute preference as to how many items should be held out; we recommend designing the evaluation methods that are similar to the actual use case.

In [37]:
from tqdm import tqdm_notebook
import numpy as np
from metrics import mean_reciprocal_rank, ndcg_at_k, precision_at_k

In [38]:
relevance = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id)
    )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']]
    relevance.append([int(x in true_items.values) for x in rec_items])

In [39]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in relevance]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in relevance]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in relevance]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in relevance]))

mean_reciprocal_rank 0.145002645054113
precision_at_5 0.11493377483443709
precision_at_10 0.10456953642384106
precision_at_25 0.08750993377483444
normalized_discounted_cumulative_gain_at_5 0.177783170055624
normalized_discounted_cumulative_gain_at_10 0.2378382874621963
normalized_discounted_cumulative_gain_at_25 0.3516037599383753


### Optional: slightly better results after deduplicating previous purchase histories

In [40]:
rel_dedup = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id)
    )
    past_items = data[data.USER_ID == user_id].ITEM_ID.values
    topk = [int(x['itemId']) for x in rec_response['itemList']]
    rec_items = [x for x in topk if x not in past_items]
    if len(rec_items) < 25:
        rec_items.extend([x for x in topk if x not in rec_items])
    rec_items = rec_items[:25]    

    rel_dedup.append([int(x in true_items.values) for x in rec_items])

In [41]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_dedup]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_dedup]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_dedup]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_dedup]))

mean_reciprocal_rank 0.18802986426679386
precision_at_5 0.16082781456953643
precision_at_10 0.14276490066225164
precision_at_25 0.08750993377483444
normalized_discounted_cumulative_gain_at_5 0.2499630328682448
normalized_discounted_cumulative_gain_at_10 0.32913706965420175
normalized_discounted_cumulative_gain_at_25 0.40842287268608907


### Try comparing with popularity baseline as a dummy recommender

In [42]:
topk = data.groupby("ITEM_ID").TIMESTAMP.count().sort_values(ascending=False).iloc[:100].index.values

In [43]:
rel_popular = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_items = topk[:25]
    rel_popular.append([int(x in true_items.values) for x in rec_items])

In [44]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_popular]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_popular]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_popular]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_popular]))

mean_reciprocal_rank 0.05072304507652334
precision_at_5 0.01980132450331126
precision_at_10 0.020066225165562918
precision_at_25 0.021735099337748344
normalized_discounted_cumulative_gain_at_5 0.043792523162000045
normalized_discounted_cumulative_gain_at_10 0.06552215768085591
normalized_discounted_cumulative_gain_at_25 0.12216764300043037


### Popularity baseline deduplicating user histories

In [45]:
rel_pop_dedup = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    past_items = data[data.USER_ID == user_id].ITEM_ID.values
    rec_items = [x for x in topk if x not in past_items]
    if len(rec_items) < 25:
        rec_items.extend([x for x in topk if x not in rec_items])
    rec_items = rec_items[:25]    
    rel_pop_dedup.append([int(x in true_items.values) for x in rec_items])

In [46]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_pop_dedup]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_pop_dedup]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_pop_dedup]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_pop_dedup]))

mean_reciprocal_rank 0.08990051372135084
precision_at_5 0.06321192052980133
precision_at_10 0.055993377483443706
precision_at_25 0.04464238410596026
normalized_discounted_cumulative_gain_at_5 0.10508158681030984
normalized_discounted_cumulative_gain_at_10 0.14086321546662076
normalized_discounted_cumulative_gain_at_25 0.21005097965654965
